In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.5 MB/s eta 0:00:00


In [25]:
import pandas as pd
import random
import pprint # 보기 쉽게 깔끔한 형태로 출력

from transformers import AutoModel, AutoTokenizer

from torch import nn # 신경망(neural network)생성
import torch.nn.functional as F # 활성화함수, 손실함수를 대체하여 코드를 간결하게 만듬

from sklearn.metrics import accuracy_score, f1_score

In [26]:
data = pd.read_excel("./data/data.xlsx", engine="openpyxl")#.iloc[:100]
# data["label_idx"] = [random.choice([0,1,2,3,4]) for _ in range(100)]
display(data.head(2))
data.shape

,sentence,label_idx
0,네,0
1,네 접시 색깔은 다른데 가격은 똑같아요,1997


(15726, 2)

In [27]:
all_indices = data.index

In [28]:
train_size = int(0.8 * len(all_indices)) # data의 70% 훈련용
val_size = len(all_indices) - train_size # data의 30% 검증용
train_idx, val_idx = all_indices[:int(0.8 * len(all_indices))], all_indices[int(0.8 * len(all_indices)):]

num_classes = max(data["label_idx"].tolist())+1 # data 개수 == 인덱스 최대값 + 1

In [29]:
train_idx, val_idx

(RangeIndex(start=0, stop=12580, step=1),
 RangeIndex(start=12580, stop=15726, step=1))

In [30]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, BertModel
from transformers import get_constant_schedule_with_warmup, get_linear_schedule_with_warmup # 학습 스케줄러를 생성
from tqdm.auto import tqdm # 프로그램 실행 진행사항 확인

# transformers 라이브러리에서 BERT 토크나이저를 임포트합니다.
from transformers import BertTokenizer

# 다국어 지원 BERT 모델을 사용하여 토크나이저 객체를 생성합니다.
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")

- BERT 입력형태
  - [CLS] 토큰 + 첫번째 문장 + [SEP] 토큰 + 두번째 문장
    - 문장의 시작과 끝, 문장 간의 구분을 명확하게 인식 가능
  - [CLS] 토큰
    - "Classification token", 입력 문장의 시작 부분에 추가.
    - 이 토큰의 임베딩은 문장 분류 작업에 사용, 문장 전체의 문맥을 요약하는 역할
  - [SEP] 토큰
    - "Separator token", 두 문장을 구분하는 데 사용
  - 임베딩
    - 텍스트 데이터를 컴퓨터가 이해하고 처리할 수 있는 수치형 벡터로 변환하는 과정 혹은 변환된 벡터


In [31]:
class TextClassificationDatasetBatch(Dataset):
    def __init__(self, df, tokenizer, batch_size=4):
        self.tokenizer = tokenizer # 토크나이저를 클래스 변수로 저장
        self.sentences = df["sentence"].tolist() # "sentence" 열의 데이터를 리스트로 변환하여 저장
        self.labels = df["label_idx"].tolist() # "label_idx" 열의 데이터를 리스트로 변환하여 저장

    def __len__(self): # 데이터 셋의 총 길이를 반환(레이블 개수 반환)
        return len(self.labels)

    def __getitem__(self, idx):
        # to_return = {
        #     "sentence_encoded":self.encodings[idx],
        #     "label":self.labels[idx]
        # }
        return (self.sentences[idx], self.labels[idx]) # 인덱스에 해당하는 문장과 레이블을 튜플로 묶어 반환

In [32]:
train_data = data.iloc[train_idx]
val_data = data.iloc[val_idx]

In [33]:
# train_dataset = TextClassificationDataset(train_data, tokenizer)
# val_dataset = TextClassificationDataset(val_data, tokenizer)
train_dataset = TextClassificationDatasetBatch(train_data, tokenizer)
val_dataset = TextClassificationDatasetBatch(val_data, tokenizer)

In [34]:
train_dataset[0]

('네', 0)

In [35]:
# DataLoader : PyTorch에서 데이터 로드하는데 사용하는 클래스
# 데이터셋을 batch_size만큼 분할하여 로드
# shuffle=True: epoch마다 데이터셋을 무작위로 섞음(과적합 방지)
# shuffle=False: 데이터셋을 섞지않음(검증 데이터는 순서가 결과에 영향을 미치지 않기때문)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# 'next', 'iter' 함수를 이용해 DataLoader 객체에서 첫 번째 배치를 가져옴
# next: 반복자의 다음 값을 반환, 모든 값 소진시 'StopIteration'예외 발생
# iter: 객체를 인자로 받아 해당 객체의 반복자(iterator)를 반환
t = next(iter(train_dataloader))

In [ ]:
t[0]

('고객님 죄송합니다만 맞쵸킹은 오늘 재료소진으로 판매가 끝났습니다',
 '네 있습니다',
 '기본적인 거 다 들어가고 깻잎도 들어가고요',
 '네 선불이에요',
 '네 아니에요',
 '3명 예약 했어요',
 '여기는 몇 시부터 문 열어요?',
 '어떤 게 잘 나가나요?',
 '일본어로 텐코쿠가 천국이라는 말이에요',
 '그것도 맛있어 보이네',
 '네 그럼 도착해서 전화주시면 1층으로 내려가서 받을께요',
 '네 맞습니다',
 '여기 예약도 되는가요?',
 '네 다 맛있어요',
 '여기는 오픈한지가 얼마 됐어요?',
 '네 오징어 좀 있으면 나올 겁니다',
 '떡볶이 1인분 순대 1인분 오뎅 세개 포장해주세요',
 '메뉴판은 어디 있나요? ',
 '올라가서 왼편 보시면 있어요',
 '중간으로 해주세요',
 '삼겹살 짜장 볶음은 삼겹살로 하는 거예요?',
 '네 치킨무 많이 주세요',
 '아직 일행이 안와서 그런데 조금 이따가 주문해도 될까요?',
 '네 뚝배기 해물뚝배기가 제일 잘 나가요',
 '세트 1개는 변경 안하고 1개만 옥수수콘으로 변경해주세요',
 '메뉴 나와있는 스티커 하나만 주세요',
 '혹시 1인분도 배달 가능한가요',
 '여기서 주문해도 되나요?',
 '마차이짬뽕으로 주세요',
 '네 섞어서 주세요',
 '짜장면 하나요',
 '등갈비찜 저런 것도 해요?')

In [ ]:
t[1]

tensor([2731,    0, 1274,   33,    0, 2055, 1750,    0, 1558,  832, 2500,    0,
         405,    0, 2332,  774, 1671,  866,  463, 2279, 1234,  523,   51,  561,
         341,  998, 2642,  536, 2880,    0, 2199, 1714])

- input_ids: 전체 입력 데이터의 token id
- attention_mask: 연산이 필요없는 패딩 토큰에 대해 어텐션을 하지 않도록 마스킹해주는 역할
- token_type_ids: 각 token이 어떤 타입에 속하는지 나타내는 역할

In [ ]:
for i, t_data in enumerate(train_dataloader):
  tokenized = tokenizer(t_data[0], return_tensors="pt", padding=True, truncation=True, max_length=512)
  print(tokenized["input_ids"].shape, model(**tokenized)["pooler_output"].shape)
  # model(**tokenized) == model(input_ids=tokenized["input_ids"], token_type_ids=tokenized["token_type_ids"], attention_mask=tokenized["attention_mask"])

In [ ]:
# tokenized.keys()

In [ ]:
# model(**tokenized)

In [46]:
import torch
import torch.nn as nn


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_classes = max(data["label_idx"].tolist())+1 # 2998
epochs = 100  # 에폭 수
batch_size = 64  # 배치 크기
learning_rate = 1e-5  # 학습률

In [47]:
def collate_bert_tokenizer(samples):
  # print("sample : ", samples)
  sentences, labels = [s[0] for s in samples], [s[1] for s in samples]
  global tokenizer
  return tokenizer(sentences, return_tensors='pt', padding=True, truncation=True, max_length=512), labels

In [48]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn = collate_bert_tokenizer)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn = collate_bert_tokenizer)

In [49]:
# nn.Module을 상속하여 사용자 정의 모델 클래스 생성
class CustomModel(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(CustomModel, self).__init__() # 부모 클래스의 생성자 호출
        # 사전 학습된 BERT 모델을 불러옴
        self.bert = BertModel.from_pretrained(bert_model)
        # 드롭아웃 레이어를 추가(과적합 방지)
        self.dropout = nn.Dropout(0.1)
        # 출력 레이어 추가(BERT 모델의 hidden_size를 입력받고, 출력 크기는 분류할 클래스의 개수)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        # 입력 데이터를 BERT 모델에 통과
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # BERT 모델의 출력 중 pooler_output(문장의 전체적인 의미)을 사용
        pooled_output = outputs.pooler_output
        # 드롭아웃 레이어 통과
        dropout_output = self.dropout(pooled_output)
        # 출력 레이어를 통과시켜 최종 출력을 반환
        return self.fc(dropout_output)

# CustomModel 클래스를 사용하여 모델을 생성합니다. BERT 모델로는 "monologg/kobert"을,
# 분류할 클래스의 개수는 num_classes(2998)를 사용합니다.
model = CustomModel("monologg/kobert", num_classes).to(device)

optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
# 학습 스케줄러를 생성(학습률을 동적으로 조정)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * epochs)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
for i, (_input, _label) in enumerate(train_dataloader):
        # 각 레이블을 원핫 인코딩으로 변환하고 텐서로 변환
        _label = torch.tensor(F.one_hot(torch.tensor(_label, dtype=torch.long), num_classes=num_classes), dtype=torch.float).squeeze()
        print(_label.shape, _input['input_ids'].shape)

In [51]:
loss_f = nn.CrossEntropyLoss()

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    print('-' * 10)
    # 모델을 학습 모드로 설정합니다.
    model.train()
    # 입력 데이터와 레이블 전처리
    total_loss = 0
    for i, (_input, _label) in enumerate(tqdm(train_dataloader, desc="Training")):
        input_ids = torch.tensor(_input['input_ids']).to(device)
        attention_mask = torch.tensor(_input['attention_mask']).to(device)
        _label = torch.tensor(_label).to(device)

        outputs = model(input_ids, attention_mask)
        loss = loss_f(outputs, _label)

        loss.backward() # 역전파를 수행합니다.
        optimizer.step() # optimizer를 사용하여 파라미터 업데이트
        optimizer.zero_grad() # 그라디언트 초기화
        scheduler.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Training Loss: {avg_train_loss}")

    # 모델을 평가 모드로 설정합니다.
    model.eval()
    all_predictions = []
    all_labels = []
    total_val_loss = 0

    for i, (_input, _label) in enumerate(tqdm(val_dataloader, desc="Validation")):
        input_ids = torch.tensor(_input['input_ids']).to(device)
        attention_mask = torch.tensor(_input['attention_mask']).to(device)
        _label = torch.tensor(_label).to(device)

        outputs = model(input_ids, attention_mask)
        loss = loss_f(outputs, _label)
        total_val_loss += loss.item()

        # 모델의 예측과 실제 레이블을 저장
        # 예측은 가장 높은 확률을 가진 클래스를 선택
        all_predictions.extend( torch.argmax(outputs, dim=1).tolist() )
        all_labels.extend(_label.tolist())

    avg_val_loss = total_val_loss / len(val_dataloader)
    print(f"Validation Loss: {avg_val_loss}")

    # 정확도와 가중치 평균 F! score 계산하여 출력
    print(f"Accuracy: {accuracy_score(all_labels, all_predictions)}")
    print(f'F1_score: {f1_score(all_labels, all_predictions, average="weighted")}')



Epoch 1/100
----------


Training:   0%|          | 0/197 [00:00<?, ?it/s]

<ipython-input-51-9b95a5b32973>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(_input['input_ids']).to(device)
<ipython-input-51-9b95a5b32973>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(_input['attention_mask']).to(device)


Training Loss: 7.127828123605796


Validation:   0%|          | 0/50 [00:00<?, ?it/s]

...

<ipython-input-51-9b95a5b32973>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(_input['input_ids']).to(device)
<ipython-input-51-9b95a5b32973>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(_input['attention_mask']).to(device)


Training Loss: 4.497608815352929


Validation:   0%|          | 0/50 [00:00<?, ?it/s]

<ipython-input-51-9b95a5b32973>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(_input['input_ids']).to(device)
<ipython-input-51-9b95a5b32973>:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(_input['attention_mask']).to(device)


Validation Loss: 5.947439861297608
Accuracy: 0.2603305785123967
F1_score: 0.14466794821136847


In [ ]:
while True:
  # 임의의 문장을 입력합니다
  test_sentence = input()
  if test_sentence == '수고하세요':
    break

  # 문장을 토큰화합니다
  inputs = tokenizer(test_sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
  if 'token_type_ids' in inputs:
    del inputs['token_type_ids']

  # 모델을 평가 모드로 설정하고 GPU로 이동
  model.eval()
  inputs = {k: v.to(device) for k, v in inputs.items()}

  # 모델에서 예측 수행
  with torch.no_grad():
      outputs = model(**inputs)
      predictions = outputs.argmax(dim=1)
      predicted_label = predictions.cpu().numpy()[0]

  # 예측된 라벨 출력
  print(f"Input Sentence: {test_sentence}")
  print(f"Predicted Label: {predicted_label}")

안녕하세요
Input Sentence: 안녕하세요
Predicted Label: 0
치킨 주문하고싶은데요
Input Sentence: 치킨 주문하고싶은데요
Predicted Label: 618
네. 메뉴를 선택해주시겠어요
Input Sentence: 네. 메뉴를 선택해주시겠어요
Predicted Label: 998
음... 간장 후라이드 반반 세트로 하고싶어요
Input Sentence: 음... 간장 후라이드 반반 세트로 하고싶어요
Predicted Label: 2432
세트메뉴로 하고싶으신건가요?
Input Sentence: 세트메뉴로 하고싶으신건가요?
Predicted Label: 2448
네. 간장치킨 세트요
Input Sentence: 네. 간장치킨 세트요
Predicted Label: 2759
네 알겠습니다.
Input Sentence: 네 알겠습니다.
Predicted Label: 0
#주소# 가 어떻게 되시나요?
Input Sentence: #주소# 가 어떻게 되시나요?
Predicted Label: 1876
#주소#로 배달해주시면 됩니다.
Input Sentence: #주소#로 배달해주시면 됩니다.
Predicted Label: 1876
네 주문해주셔서 감사합니다.
Input Sentence: 네 주문해주셔서 감사합니다.
Predicted Label: 536
수고하세요
